# Monads in Julia: TypeClasses.jl and DataTypesBasic.jl

An implementation of standard Monads in Julia.
* https://github.com/JuliaFunctional/TypeClasses.jl: definition of standard interfaces + standard implementation 
* https://github.com/JuliaFunctional/DataTypesBasic.jl: definition of standard types

<img src="images/jakobsweg-fisterra-fokus.png"
     alt="Stephan Sahm"
     style="float: right; margin-right: 3em"
     width="62%"/>

### Stephan Sahm
- freelancer
- end-to-end Data & AI consultant
- organizer of Julia User Group Munich

### key interests
- professional best practices
- probabilistic programming
- functional programming

### you are always welcome to reach out
- github: schlichtanders
- mail: stephan.sahm@gmx.de
- linkedin: https://de.linkedin.com/in/stephan-sahm-918656b7


# TypeClasses.jl

TypeClasses.jl defines standard functional interfaces which are widely used.

------------------

# Design Principles

* reuse as much `Base` as possible
* make it stable (hence so far we only support the most important type-classes)
* make it simple
* make it convenient

# Functor/Applicative/Monad

Typeclass | Interface | Helpers from `TypeClasses`
--------- | --------- | --------------------------
          | `TypeClasses.foreach = Base.foreach` | `@syntax_foreach`
Functor, Applicative, Monad  | `TypeClasses.map = Base.map` | `@syntax_map`
Applicative, Monad | `TypeClasses.pure`, `TypeClasses.ap` | `mapn`, `@mapn`, `tupled`, `neutral_applicative`, `combine_applicative`, `orelse_applicative`
Monad | `TypeClasses.flatmap` | `flatten`, `↠` (\twoheadrightarrow),  `@syntax_flatmap`

We decided to use `flatmap` as the interface, because it is often more intuitiv to implement than `flatten` and also comes quite natural next to `map`.

In order to enable simple interactions between monads, all `flatmap` implementations use `convert` before flattening.

`@syntax_flatmap` provides monadic syntax (similar to haskell do-notation). However, the macro translates to `flatmap` and `map` only, and does not need `pure`.

`mapn` is explicitly an extra function, because it has a generic definition which uses `pure` and `ap` and should not be confused with standard `map` in terms of Functor to avoid method ambiguation errors.

Each Applicative can lift an underlying Monoid. In addition some Applicatives also define Monoids themselves (e.g. Vector). Hence, we distinguish both by adding functions `neutral_applicative`, `combine_applicative`, `orelse_applicative`.

You can overload `TypeClasses.map` or `Base.map`, as you like, they are both the very same.

# Semigroup/Monoid/Alternative

Typeclass | Interface | Helpers from `TypeClasses`
--------- | --------- | --------------
Monoid, Alternative | `TypeClasses.neutral` | 
Semigroup, Monoid | `TypeClasses.combine` | alias `⊕` (\oplus), `reduce_monoid`, `foldr_monoid`, `foldl_monoid`
Alternative | `TypeClasses.orelse` | alias `⊘` (\oslash)

<br>

-------------

We decided to use the same `neutral` for both Monoid and Alternative because of simplicity. 

Julia does not have stable typeparameters (for optimization a typeparameter may be inferred as Any instead of more concrete type), and hence Alternative (which is concept targeted at Functors, i.e. things with one typeparameter) becomes way more similar to Monoid.

# FlipTypes

Typeclass | Interface | Helpers from `TypeClasses`
--------- | --------- | --------------------------
FlipTypes | `TypeClasses.flip_types` | `TypeClasses.default_flip_types_having_pure_combine_apEltype`

<br>

---------

`flip_types(::A{B{C}})` should return `::B{A{C}}`. Hence the name: it flips the first two types. 

FlipTypes is not an official TypeClass, however proofs to be a very essential abstraction. Normally this comes with the TypeClass Traversable and is called `sequence`, however that name is not very self-explanatory and sounds quite specific.

`TypeClasses.flip_types` has already one big usage in `ExtensibleEffects.jl`, for a generic implementation of effect handling.

# DataTypesBasic.jl

Defines standard types typically used in functional programming. 

**Design principles:** stable, simple, and convenient.

# Option, Try, Either

* `Option`: something which can be `nothing` (name taken from Scala)
* `Try`: something which can error (name taken from Scala)
* `Either`: something which can short-cycle (name taken from Haskell/Scala)

All three are very similar to one another in behaviour. They all define some kind of stopping behaviour and possibly return further information why to stop.

# Identity, Const

After trying several different approaches of creating new type hierarchies for each separately, we simplified the implementation to reuse the same building blocks for all three.

The core are two types:
* `Identity`: denoting normal behaviour
* `Const`: denoting a stopping action with possible extra information

The names are the same as respective Haskell types, which also have the very same behaviour when used separately. 

Unlike Haskell, this Julia implementation also defines interactions between `Identity` and `Const`. These interactions, together with `Union` types then define `Option`, `Try`, and `Either`.

# All Data Types from DataTypesBasic.jl

DataType | Implementation | Helpers
-------- | -------------- | -------
ContextManager | `ContextManager`, encapsulating construction and destruction, aka enter and exit. | call syntax, `Base.run`, `Base.eltype`, `Base.foreach`, `Base.map`, `Base.Iterators.flatten`, only Functor/Applicative/Monad TypeClasses
Identity | `Identity`     | `isidentity`, `Base.length`, `Base.get`, `Base.getindex`, `Base.iterate`, `Base.foreach`, `Base.map`, `Base.Iterators.flatten`, `Base.eltype`, `Base.convert`, `Base.promote_type`, `Base.promote_typejoin`, all&nbsp;TypeClasses
Const    | `Const`        |  `Base.isconst`, `Base.length`, `Base.iterate`, `Base.foreach`, `Base.map`, `Base.Iterators.flatten`, `Base.eltype`, `Base.convert`, `Base.promote_type`, `Base.promote_typejoin`, almost all TypeClasses, but without `pure`
Either   | `Either{L, R} = Union{Const{L}, Identity{R}}` | `Either`, `Base.eltype`, `either`, `@either`, `flip_left_right`, `iseither`, `isleft`, `isright`, `getleft`, `getright`, `getleftOption`, `getrightOption`, `getOption`, all&nbsp;TypeClasses
Try      | `Try{T} = Union{Const{<:Exception}, Identity{T}}` | `Try`, `@Try`, `@TryCatch`,  `MultipleExceptions`, `Base.eltype`, `istry`, `issuccess`, `isfailure`, all&nbsp;TypeClasses
Option   | `Option{T} = Union{Const{Nothing}, Identity{T}}` | `Option`, `isoption`, `issome`, `isnone`, `Base.eltype`, `iffalse`, `iftrue`, all&nbsp;TypeClasses

# All Data Types from TypeClasses.jl

TypeClasses.jl defines some further standard data types as examples.

DataType | Implementation | Helpers
-------- | -------------- | -------
DataTypesBasic.jl | everything is re-exported | everything is re-exported
Callable | `Callable`, simple wrapper type | call syntax, `combine`, `orelse`, `map`, `pure`, `ap`, `flatmap`, no Base.eltype, no neutral, no flip_types
Iterable | `Iterable`, simple wrapper type | whole iterator interface, including Base.IteratorEltype, Base.eltype and others, `Base.convert`, `neutral`, `combine`, `foreach`, `map`, `pure`, `ap`, `flatmap`, `flip_types`
State    | `State`, wrapper around function s&nbsp;->&nbsp;(a,&nbsp;s) | call syntax, `getstate`, `putstate`, `combine`, `orelse`, `map`, `pure`, `ap`, `flatmap`, no Base.eltype, no neutral, no flip_types
Writer | `Writer{Acc, Value}`, similar to Base.Pair where `Acc` supports `combine` | `Base.eltype`, `foreach`, `map`, `pure`, `ap`, `flatmap`, `flip_types`

# Let's see how they work

<center>
    
## Option

</center>

In [1]:
using TypeClasses

Option(3), Option(nothing)

(Identity(3), Const(nothing))

In [2]:
@syntax_flatmap begin
    a = Option(1)
    iftrue(isodd(a)) do
        div(a - 1, 2)
    end
end

Identity(0)

In [3]:
@syntax_flatmap begin
    a = Option(2)
    isodd(a) ? Option(div(a - 1, 2)) : Option()
end



Const(nothing)

<center>
    
## Try

</center>

In [4]:
success = @Try "all-fine"
failure = @Try error("wrong")
success, failure

(Identity("all-fine"), Const(Thrown(ErrorException("wrong"))))

In [5]:
failure[]

Thrown(ErrorException("wrong"))
wrong
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:33
  [2] top-level scope
    @ ~/.julia/dev/DataTypesBasic/src/Try.jl:119
  [3] eval
    @ ./boot.jl:360 [inlined]
  [4] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
    @ Base ./loading.jl:1094
  [5] softscope_include_string(m::Module, code::String, filename::String)
    @ SoftGlobalScope ~/.julia/packages/SoftGlobalScope/u4UzH/src/SoftGlobalScope.jl:65
  [6] execute_request(socket::ZMQ.Socket, msg::IJulia.Msg)
    @ IJulia ~/.julia/packages/IJulia/e8kqU/src/execute_request.jl:67
  [7] #invokelatest#2
    @ ./essentials.jl:708 [inlined]
  [8] invokelatest
    @ ./essentials.jl:706 [inlined]
  [9] eventloop(socket::ZMQ.Socket)
    @ IJulia ~/.julia/packages/IJulia/e8kqU/src/eventloop.jl:8
 [10] (::IJulia.var"#15#18")()
    @ IJulia ./task.jl:411


<center>
    
## Either

</center>

In [6]:
left = Either{String}("left")
right = Either{String}(:right)
left, right

(Const("left"), Identity(:right))

In [7]:
@either true ? :right : "left"

Identity(:right)

In [8]:
@syntax_flatmap begin
    a = either("left", false, "right")
    @Try error("a = $a")
end

Const("left")

# Composing Monads

⚠️ Monads can only compose when **later monads can be converted to former ones**.

😀 As `Either`, `Try`, and `Option` all are implemented via `Identity` and `Const`, they compose naturally.

😀 Composing arbitrary Monads can actually be done using [ExtensibleEffects.jl](https://github.com/JuliaFunctional/ExtensibleEffects.jl)

In [9]:
using TypeClasses  # also loads DataTypesBasic

@syntax_flatmap begin
    (a, b) = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
    # constructs an Option
    c = iftrue(a % 2 == 0) do
        a + b
    end
    # constructs an Either
    d = either("left", b < 4, :right)
    # constructs an Try, throws DivideError
    e = @Try div(b, a)
    @pure "a = $a, b = $b, c = $c, d = $d, e = $e" 
end

1-element Vector{Any}:
 "a = 2, b = 3, c = 5, d = right, e = 1"

In [10]:
@syntax_flatmap begin
    a = Option(5)
    b = Iterable(1:a)
    @pure (a,b)
end

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mIterable{Vector{Tuple{Int64, Int64}}}[39m[0m to an object of type 
[0m  [91mIdentity[39m
[0mClosest candidates are:
[0m  convert(::Type{var"#s9"} where var"#s9"<:Identity, [91m::ContextManager[39m) at /home/ssahm/.julia/dev/DataTypesBasic/src/convert.jl:28
[0m  convert(::Type{Identity}, [91m::Const[39m) at /home/ssahm/.julia/dev/DataTypesBasic/src/convert.jl:26
[0m  convert(::Type{var"#s28"} where var"#s28"<:Identity, [91m::Writer[39m) at /home/ssahm/.julia/dev/TypeClasses/src/convert.jl:10
[0m  ...